# Tree-based classifier

Include all possible dimensions and train a treebased classifier

In [1]:
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics
from sklearn import utils
from sklearn.tree import DecisionTreeClassifier

In [88]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import export_graphviz
import graphviz


## Load Data

In [3]:
df = pd.read_csv('/Users/leima/data_store/rideindego_all.csv')

/Users/leima/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,3,4,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df['date'] = pd.to_datetime(df.date)

In [5]:
df.head()

,bike_id,bike_type,duration,end_lat,end_lon,end_station_id,end_time,passholder_type,plan_duration,start_lat,start_lon,start_station_id,start_time,trip_id,trip_route_category,date,hour,weekday,month
0,3640,standard,4.0,39.9384,-75.173271,3064,7/1/2016 0:11,Indego30,30,39.946331,-75.1698,3063.0,7/1/2016 0:07,25183529,One Way,2016-07-01,0,4,7
1,3349,standard,8.0,39.94138,-75.145638,3026,7/1/2016 0:18,Indego30,30,39.95295,-75.139793,3124.0,7/1/2016 0:10,25183528,One Way,2016-07-01,0,4,7
2,5279,standard,17.0,39.933151,-75.162483,3034,7/1/2016 0:28,Indego30,30,39.93082,-75.174744,3043.0,7/1/2016 0:11,25183527,One Way,2016-07-01,0,4,7
3,5198,standard,25.0,39.94138,-75.145638,3026,7/1/2016 0:38,Walk-up,0,39.94138,-75.145638,3026.0,7/1/2016 0:13,25183526,Round Trip,2016-07-01,0,4,7
4,3382,standard,45.0,39.947109,-75.166183,3010,7/1/2016 0:58,Walk-up,0,39.954239,-75.161377,3037.0,7/1/2016 0:13,25183525,One Way,2016-07-01,0,4,7


In [6]:
df['duration_original'] = df.duration

In [7]:
df['duration'] = df.duration.apply(lambda x: int(x/5)*5 )

In [8]:
df.head()

,bike_id,bike_type,duration,end_lat,end_lon,end_station_id,end_time,passholder_type,plan_duration,start_lat,start_lon,start_station_id,start_time,trip_id,trip_route_category,date,hour,weekday,month,duration_original
0,3640,standard,0,39.9384,-75.173271,3064,7/1/2016 0:11,Indego30,30,39.946331,-75.1698,3063.0,7/1/2016 0:07,25183529,One Way,2016-07-01,0,4,7,4.0
1,3349,standard,5,39.94138,-75.145638,3026,7/1/2016 0:18,Indego30,30,39.95295,-75.139793,3124.0,7/1/2016 0:10,25183528,One Way,2016-07-01,0,4,7,8.0
2,5279,standard,15,39.933151,-75.162483,3034,7/1/2016 0:28,Indego30,30,39.93082,-75.174744,3043.0,7/1/2016 0:11,25183527,One Way,2016-07-01,0,4,7,17.0
3,5198,standard,25,39.94138,-75.145638,3026,7/1/2016 0:38,Walk-up,0,39.94138,-75.145638,3026.0,7/1/2016 0:13,25183526,Round Trip,2016-07-01,0,4,7,25.0
4,3382,standard,45,39.947109,-75.166183,3010,7/1/2016 0:58,Walk-up,0,39.954239,-75.161377,3037.0,7/1/2016 0:13,25183525,One Way,2016-07-01,0,4,7,45.0


In [9]:
df = df[df.bike_type == 'standard']
coi = [
    'passholder_type', 
    'trip_route_category', 'hour', 'weekday', 'month'
]
cot = ['duration']
df = df[coi+cot]

In [10]:
df.head()

,passholder_type,trip_route_category,hour,weekday,month,duration
0,Indego30,One Way,0,4,7,0
1,Indego30,One Way,0,4,7,5
2,Indego30,One Way,0,4,7,15
3,Walk-up,Round Trip,0,4,7,25
4,Walk-up,One Way,0,4,7,45


In [11]:
for i in df.columns:
    print(
        i, ': ',
        df[i].isnull().any()
    )

passholder_type :  False
trip_route_category :  False
hour :  False
weekday :  False
month :  False
duration :  False


In [12]:
scaler = StandardScaler()

In [62]:
X = df[coi]
#X = pd.get_dummies(X)
y = df[cot]
# y = scaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3, random_state=42
    #,stratify=Y
)

In [14]:
class MultiColumnLabelEncoder:
    def __init__(self, encoders = None, columns = None):
        self.columns = columns # array of column names to encode
        if encoders:
            self.encoders = encoders
        else:
            self.encoders = {}

    def fit(self,X,y=None):
        self.check_encoders = []
        return self # not relevant here

    def transform(self, X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                _le = LabelEncoder()
                if self.encoders.get(col):
                    output[col] = self.encoders.get(col).transform(output[col])
                else:
                    output[col] = _le.fit_transform(output[col])
                    print(f'1. preparing encoder for {col}')
                    self.encoders[col] = _le
                    self.check_encoders.append({col: _le})
        else:
            for colname,col in output.iteritems():
                _le = LabelEncoder()
                if self.encoders.get(col):
                    output[colname] = self.encoders.get(col).transform(col)
                else:
                    output[colname] = _le.fit_transform(col)
                    print(f'2. preparing encoder for {col}')
                    self.encoders[col] = _le
                    self.check_encoders.append({col: _le})
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [52]:
encoders_obj = MultiColumnLabelEncoder(columns=coi+['duration'])
encoders_obj.fit_transform(df)
dt_encoders = encoders_obj.encoders

1. preparing encoder for passholder_type
1. preparing encoder for trip_route_category
1. preparing encoder for hour
1. preparing encoder for weekday
1. preparing encoder for month
1. preparing encoder for duration


In [16]:
coi

['passholder_type', 'trip_route_category', 'hour', 'weekday', 'month']

In [40]:
dt_classifier = GridSearchCV(
    DecisionTreeClassifier(random_state=42), cv=3, 
    param_grid={"max_depth": [1,2,3,4,5,6,7,8]}
)

pipeline_steps = [
    ('encoding', MultiColumnLabelEncoder(columns=coi, encoders=dt_encoders)),
    ('dt_classifier', dt_classifier)
    # add more pipeline steps as needed
]

In [41]:
model = Pipeline(pipeline_steps)

In [78]:
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('encoding', <__main__.MultiColumnLabelEncoder object at 0x1a143865c0>), ('dt_classifier', GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_i...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))])

In [79]:
model.score(X_test[:20000], y_test[:20000])

0.31815

In [44]:
model.score(X_train, y_train)

0.3209558739900585

In [64]:
X_train_transformed = pd.DataFrame()
for col in X_train.columns:
    X_train_transformed[col] = dt_encoders.get(col).transform(X_train[col].values)
    print(col)

passholder_type
trip_route_category
hour
weekday
month


In [65]:
y_train_transformed = pd.DataFrame()
for col in y_train.columns:
    y_train_transformed[col] = dt_encoders.get(col).transform(y_train[col].values)
    print(col)

duration


In [92]:
print(
    classification_report(y_test, model.predict(X_test))
)

             precision    recall  f1-score   support

          0       0.30      0.25      0.27     62950
          5       0.35      0.94      0.51    233852
         10       0.21      0.07      0.10    194315
         15       0.16      0.06      0.09    105568
         20       0.19      0.00      0.01     57686
         25       0.11      0.02      0.04     33605
         30       0.00      0.00      0.00     20469
         35       0.00      0.00      0.00     13529
         40       0.00      0.00      0.00     10071
         45       0.00      0.00      0.00      8444
         50       0.00      0.00      0.00      7420
         55       0.00      0.00      0.00      6252
         60       0.00      0.00      0.00      4064
         65       0.00      0.00      0.00      3086
         70       0.00      0.00      0.00      2540
         75       0.00      0.00      0.00      2318
         80       0.00      0.00      0.00      2018
         85       0.00      0.00      0.00   

/Users/leima/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
pipeline.named_steps['pca']

In [95]:
dt_classifier.best_params_

{'max_depth': 6}

In [110]:
dt_classifier_6 = DecisionTreeClassifier(random_state=42, max_depth=6)

dt_classifier_6.fit(X_train_transformed[:1000], y_train_transformed[:1000])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [158]:
for name, importance in zip(X_train.columns, dt_classifier_6.feature_importances_):
    print(name, f'{importance:.2}')

passholder_type 0.19
trip_route_category 0.14
hour 0.33
weekday 0.16
month 0.18


In [140]:
export_graphviz(
    dt_classifier_6, 
    feature_names=X_train.columns,
    out_file='dt_classifier_6.dot'
)

In [154]:
graph = graphviz.Source(
    export_graphviz(
    dt_classifier_6, 
    feature_names=X_train.columns,
    out_file=None
)
)

In [155]:
graph.format = 'png'

In [156]:
graph.render('dt_classifier_6.dot')

'dt_classifier_6.dot.png'

### Tests

In [ ]:
df_test = X_test.sample(n=2)

In [ ]:
df_test.info()

In [ ]:
model.predict(df_test)

In [ ]:
'{}'.format(model.predict(df_test))

In [ ]:
scaler.inverse_transform(model.predict(df_test))

In [ ]:
df_test = X_test.sample(n=1)

In [ ]:
df_test.to_json(orient='records')

In [ ]:
record = json.loads('{"passholder_type":"Indego30","trip_route_category":"One Way","hour":15,"weekday":3,"month":6}')

In [ ]:
df_test_recon = pd.DataFrame.from_records([record])

In [ ]:
df_test_recon